In [1]:
import sympy as sp

import sys,os,subprocess
sys.path.insert(0,'/work/JAM/apps/bin')
os.environ["PATH"]='/work/JAM/apps/bin:'+os.environ["PATH"]
from qgraf import QGRAF
from form import FORM,to_sympy,run_form,run_form2,to_sympy
from tools import checkdir,save,load,lprint,lpprint,evalf

import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import mathematica_code as mcode
from sympy.utilities.lambdify import lambdastr

In [2]:
code = '''
Off Statistics;
Autodeclare Index o;
Index s;

L [gamma5] = -I_*g5_(s);
L [test]   = g_(s,o1)*g_(s,o2)*g_(s,o3)*g_(s,o4)*[gamma5];

Trace4,s;

Print [test];
'''
data = run_form(code,verb=3)

[test] =
- 4*e_(o1,o2,o3,o4)*i_;


In [3]:
code = '''
Off Statistics;
Autodeclare Index o;

L [test] = e_(o1,o2,o3,o4)*e_(o1,o2,o3,o4);
contract;

Print [test];
'''
data = run_form(code,verb=3)

[test] =
24;


In [3]:
code = '''
Off Statistics;
Autodeclare Index o;

L [test] = e_(o1,o2,o3,o4)*e_(o1,o2,o5,o6);
contract;

Print [test];
'''
data = run_form(code,verb=3)

[test] =
2*d_(o3,o5)*d_(o4,o6) - 2*d_(o3,o6)*d_(o4,o5);


In [5]:
code = '''
Off Statistics;
Autodeclare Index o;
CFunction e(antisymmetric);

L [test] = e(o1,o2,o3,o4)*e(o1,o2,o5,o6);

id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));

Print [test];
'''
data = run_form(code,verb=3)

[test] =
- 2*d_(o3,o5)*d_(o4,o6) + 2*d_(o3,o6)*d_(o4,o5);


# Derive Leptonic Tensor Forms

In [6]:
code = '''
Off Statistics;
AutoDeclare Index o;
Index s;
Vector l,lp;
Symbol lam,m,gV,gA;
Cfunction dot;
Autodeclare Vector V;

L [gamma]   = (1+lam*(-I_*g5_(s)))*(g_(s,l)+m)*g_(s,o1)*(g_(s,lp)+m)*g_(s,o2)/2;
L [gamma,Z] = (1+lam*(-I_*g5_(s)))*(g_(s,l)+m)*g_(s,o1)*(g_(s,lp)+m)*g_(s,o2)*(gV-gA*(-I_*g5_(s)))/2;
L [Z]       = (1+lam*(-I_*g5_(s)))*(g_(s,l)+m)*(gV-gA*(-I_*g5_(s)))*g_(s,o1)*(g_(s,lp)+m)*g_(s,o2)*(gV-gA*(-I_*g5_(s)))/2;

*contract;
Trace4,s;

print [gamma];
print [gamma,Z];
print [Z];
'''
data = run_form(code,verb=3)
# to_sympy(data['temp'].replace(r'S','s')).subs(r'lam',sp.symbols('\lambda')).simplify()

[gamma] =
2*l(o1)*lp(o2) + 2*l(o2)*lp(o1) + 2*d_(o1,o2)*m^2 - 2*d_(o1,o2)*l.lp + 2
*e_(l,lp,o1,o2)*i_*lam;
[gamma,Z] =
2*l(o1)*lp(o2)*gV + 2*l(o1)*lp(o2)*lam*gA + 2*l(o2)*lp(o1)*gV + 2*l(o2)*
lp(o1)*lam*gA + 2*d_(o1,o2)*m^2*gV + 2*d_(o1,o2)*lam*m^2*gA - 2*
d_(o1,o2)*l.lp*gV - 2*d_(o1,o2)*l.lp*lam*gA - 2*e_(l,lp,o1,o2)*i_*gA + 2
*e_(l,lp,o1,o2)*i_*lam*gV;
[Z] =
2*l(o1)*lp(o2)*gA^2 + 2*l(o1)*lp(o2)*gV^2 + 2*l(o2)*lp(o1)*gA^2 + 2*
l(o2)*lp(o1)*gV^2 - 2*d_(o1,o2)*m^2*gA^2 + 2*d_(o1,o2)*m^2*gV^2 + 4*
d_(o1,o2)*lam*m^2*gV*gA - 2*d_(o1,o2)*l.lp*gA^2 - 2*d_(o1,o2)*l.lp*gV^2
+ 2*e_(l,lp,o1,o2)*i_*lam*gA^2 + 2*e_(l,lp,o1,o2)*i_*lam*gV^2;


# Derive Cross Section

In [40]:
code = '''
Off Statistics;
Symbols M,Q,lam,E,h;
Symbols x,y;
Symbols n;
Vectors P,q,S;
Vectors Phat,Shat;
Vectors l,lp;
Autodeclare Vector V;
Tensor ghat;
Autodeclare Index o;
Autodeclare Symbols F,g;
Cfunction dot,e(antisymmetric);

L [L] = 2*(l(o1)*lp(o2) + lp(o1)*l(o2) - l.lp*d_(o1,o2) - I_*lam*e(o1,o2,o3,o4)*l(o3)*lp(o4));
L [W] = -(d_(o1,o2) - q(o1)*q(o2)/q^2)*F1 + (P(o1) - P.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2))*F2/P.q - I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*F3/2/P.q;

L [LW] = [L]*[W];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id lp = l - q;
id P.S = 0;
id P.P = M^2;
id S.l = -h*E*M;*Q^2/2/x/y;
id S.q = -h*E*M*(y+Q^2/2/E^2);*-h*Q^2/2/x*(1+2*x^2*y*M^2/Q^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id l.l = 0;
id q.q = -Q^2;
id P.l = P.q/y;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id q.l = -Q^2/2;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^n;
id V1?.V2?^-n? = (dot(V1,V2))^-n;

print [LW];
'''
data = run_form(code,verb=0)
print(data['LW'])
data['LW'] = data['LW'].replace(r'Q','q').replace('E','e').replace('S','s')
data['LW'] = data['LW'].replace(r'sqrt_','sqrt')
data['LW'] = to_sympy(data['LW'])

lam  = sp.symbols(r'\lambda')
h    = sp.symbols(r'h')
M    = sp.symbols(r'M')
Q    = sp.symbols(r'Q')
E    = sp.symbols(r'E')
x    = sp.symbols(r'x')
y    = sp.symbols(r'y')
alfa = sp.symbols(r'\alpha')
F1,F2,F3       = sp.symbols('F_1 F_2 F_3')
g1,g2,g3,g4,g5 = sp.symbols('g_1 g_2 g_3 g_4 g_5')


data['LW'] = data['LW'].subs(r'lam',lam)
data['LW'] = data['LW'].subs('q',Q).subs('e',E)
data['LW'] = data['LW'].subs(sp.Symbol('F1'),F1).subs(sp.Symbol('F2'),F2).subs(sp.Symbol('F3'),F3)
data['LW'] = data['LW'].subs(sp.Symbol('g1'),g1).subs(sp.Symbol('g2'),g2).subs(sp.Symbol('g3'),g3).subs(sp.Symbol('g4'),g4).subs(sp.Symbol('g5'),g5)

2*Q^2*x^-1*y^-2*F2 - 2*Q^2*x^-1*y^-1*F2 + 2*Q^2*F1 - 2*Q^2*h*x^-1*y^-2*g4 + 2*Q^2*h*x^-1*y^-1*g4 - 2*Q^2*h*g5 - 2*Q^2*lam*y^-1*F3 + Q^2*lam*F3- 4*Q^2*lam*h*y^-1*g1 + 2*Q^2*lam*h*g1 - 2*M^2*x*F2 - 4*M^2*h*x*y^-1*g4+ 4*M^2*h*x*y^-1*g3 + 6*M^2*h*x*g4 - 4*M^2*h*x*g3 - 4*M^2*h*x^2*y*g5 +8*M^2*lam*h*x^2*g2 + 4*M^2*lam*h*x^2*y*g1 + 4*M^4*Q^-2*h*x^3*y*g4 - 4*M^4*Q^-2*h*x^3*y*g3


### Unpolarized part

In [43]:
upol = (data['LW'].subs(h,-1) + data['LW'].subs(h,1))/sp.S(2)

factor1  = 2*sp.pi*y*alfa**2/Q**4
factor2 = 4*sp.pi*alfa**2/x/y/Q**2

factor = factor1/factor2
# display(factor)

coeff1 = factor*upol.subs(F1,1).subs(F2,0).subs(F3,0)#.subs(M,0)
coeff2 = factor*upol.subs(F1,0).subs(F2,1).subs(F3,0)#.subs(M,0)
coeff3 = factor*upol.subs(F1,0).subs(F2,0).subs(F3,1)#.subs(M,0)

display(coeff1.simplify())
display(coeff2.simplify())
display(coeff3.simplify())

x*y**2

-M**2*x**2*y**2/Q**2 - y + 1

\lambda*x*y*(y - 2)/2

# Derive Projectors

In [15]:
code = '''
Off Statistics;
Symbols M,Q,lam,h;
Symbols x,y;
Symbols n;
Vectors P,q,S;
Vectors Phat,Shat;
Autodeclare Vector V;
Autodeclare Index o;
Autodeclare Symbols F;
Cfunction dot,e(antisymmetric);

L [Wu] = -(d_(o1,o2) + q(o1)*q(o2)/Q^2)*F1 + (P(o1) + P.q/Q^2*q(o1))*(P(o2) + P.q/Q^2*q(o2))*F2/P.q - I_*e(o1,o2,o3,o4)*q(o3)*P(o4)*F3/2/P.q;

L [Pg]   = d_(o1,o2)*[Wu];
L [Ppp]  = P(o1)*P(o2)*[Wu];
L [Pepq] = I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*[Wu];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id P.P = M^2;
id q.q = -Q^2;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^(n);

print [Pg];
print [Ppp];
print [Pepq];
'''
data = run_form(code,verb=0)

# lam  = sp.symbols(r'\lambda')
# h    = sp.symbols(r'h')
M = sp.symbols(r'M')
Q = sp.symbols(r'Q')
x = sp.symbols(r'x')
y = sp.symbols(r'y')
F1,F2,F3 = sp.symbols('F_1 F_2 F_3')

for _ in data:
    data[_] = data[_].replace(r'Q','q')
    data[_] = data[_].replace(r'sqrt_','sqrt')
    data[_] = to_sympy(data[_])
    data[_] = data[_].subs('q',Q)
    data[_] = data[_].subs('M',M)
    data[_] = data[_].subs('F1',F1).subs('F2',F2).subs('F3',F3)
#     data[_] = data[_].simplify().factor(F1,F2,F3)
#     display(data[_])

$F_1$ projection

In [16]:
a1g,a1pp,a1epq = sp.symbols(r'a_{1}^{g} a_{1}^{pp} a_{1}^{epq}')
expr = a1g*data['Pg'] + a1pp*data['Ppp'] + a1epq*data['Pepq']

eq1 = sp.Eq(expr.subs(F1,1).subs(F2,0).subs(F3,0),1)
eq2 = sp.Eq(expr.subs(F1,0).subs(F2,1).subs(F3,0),0)
eq3 = sp.Eq(expr.subs(F1,0).subs(F2,0).subs(F3,1),0)

sol = sp.solve([eq1,eq2,eq3],[a1g,a1pp,a1epq])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()#.subs(M,0)
    display(sp.Eq(_,sol[_]))

Eq(a_{1}^{g}, -1/2)

Eq(a_{1}^{pp}, 2*x**2/(4*M**2*x**2 + Q**2))

Eq(a_{1}^{epq}, 0)

$F_2$ projection

In [17]:
a2g,a2pp,a2epq = sp.symbols(r'a_{2}^{g} a_{2}^{pp} a_{2}^{epq}')
expr = a2g*data['Pg'] + a2pp*data['Ppp'] + a2epq*data['Pepq']

eq1 = sp.Eq(expr.subs(F1,1).subs(F2,0).subs(F3,0),0)
eq2 = sp.Eq(expr.subs(F1,0).subs(F2,1).subs(F3,0),1)
eq3 = sp.Eq(expr.subs(F1,0).subs(F2,0).subs(F3,1),0)

sol = sp.solve([eq1,eq2,eq3],[a2g,a2pp,a2epq])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()#.subs(M,0)
    display(sp.Eq(_,sol[_]))

Eq(a_{2}^{g}, -Q**2*x/(4*M**2*x**2 + Q**2))

Eq(a_{2}^{pp}, 12*Q**2*x**3/(4*M**2*x**2 + Q**2)**2)

Eq(a_{2}^{epq}, 0)

$F_3$ projection

In [18]:
a3g,a3pp,a3epq = sp.symbols(r'a_{3}^{g} a_{3}^{pp} a_{3}^{epq}')
expr = a3g*data['Pg'] + a3pp*data['Ppp'] + a3epq*data['Pepq']

eq1 = sp.Eq(expr.subs(F1,1).subs(F2,0).subs(F3,0),0)
eq2 = sp.Eq(expr.subs(F1,0).subs(F2,1).subs(F3,0),0)
eq3 = sp.Eq(expr.subs(F1,0).subs(F2,0).subs(F3,1),1)

sol = sp.solve([eq1,eq2,eq3],[a3g,a3pp,a3epq])
sol
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()#.subs(M,0)
    display(sp.Eq(_,sol[_]))

Eq(a_{3}^{g}, 0)

Eq(a_{3}^{pp}, 0)

Eq(a_{3}^{epq}, 2*x/(4*M**2*x**2 + Q**2))

# Calculate Hadronic tensor directly

In [24]:
code = '''
Off Statistics;
Autodeclare Symbols a,v;
Autodeclare Index o;
Vector S,q,P;
Vector p,p1;
Symbol x,M,Q;
Symbol n;
CFunction e(antisymmetric);

L [Whatu] = 2*(a1*a2+v1*v2)*(p(o1)*p1(o2) + p1(o1)*p(o2) - d_(o1,o2)*p.p1) + 2*(v1*a2+a1*v2)*I_*e(o1,o2,o3,o4)*p(o3)*p1(o4);

L [Pg]   = d_(o1,o2)*[Whatu];
L [Ppp]  = P(o1)*P(o2)*[Whatu];
L [Pepq] = I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*[Whatu];

L [P1] = -[Pg]/2 + 2*x^2/(4*M^2*x^2+Q^2)*[Ppp];
L [P2] = -Q^2*x/(4*M^2*x^2+Q^2)*[Pg] + 12*Q^2*x^3/(4*M^2*x^2+Q^2)^2*[Ppp];
L [P3] = 2*x/(4*M^2*x^2+Q^2)*I_*e(o1,o2,q,P)*[Whatu];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id p1 = p + q;
id p = x*P;
id P.P = M^2;
id q.q = -Q^2;
id P.q = Q^2/2/x;
*id V1?.V2? = dot(V1,V2);

Print [P1];
Print [P2];𝐹2=2𝑥((𝑔𝑖𝐴)2+(𝑔𝑖𝑉)2)Σ𝑖𝑓𝑖
Print [P3];
'''
data = run_form(code,verb=0)

M = sp.symbols(r'M')
Q = sp.symbols(r'Q')
x = sp.symbols(r'x')
y = sp.symbols(r'y')
xi = sp.symbols(r'\xi')
F1,F2,F3 = sp.symbols('F_1 F_2 F_3')
a1,a2 = sp.symbols(r'a_(1:3)')
v1,v2 = sp.symbols(r'v_(1:3)')

for _ in data:
    data[_] = data[_].replace(r'Q','q')
    data[_] = data[_].replace(r'sqrt_','sqrt')
    data[_] = to_sympy(data[_])
    data[_] = data[_].subs(r'a1',a1).subs(r'a2',a2).subs(r'v1',v1).subs(r'v2',v2)
    data[_] = data[_].subs('q',Q)/Q**2
    data[_] = data[_].subs('M',0)
#     data[_] = data[_].subs('F1',F1).subs('F2',F2).subs('F3',F3)
    display(data[_].simplify().factor())

a_1*a_2 + v_1*v_2

2*x*(a_1*a_2 + v_1*v_2)

2*(a_1*v_2 + a_2*v_1)

In [25]:
gV   = sp.symbols(r'g_{V}^{i}')
gA   = sp.symbols(r'g_{A}^{i}')
f    = sp.Symbol(r'f_i',commutative=False)
sig  = sp.Symbol(r'\Sigma_{i}',commutative=False)
eq   = sp.Symbol(r'e_{q}',commutative=False)

$\gamma$

In [29]:
display(sp.Eq(F1,sig*eq**2*data['P1'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
display(sp.Eq(F2,sig*eq**2*data['P2'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
display(sp.Eq(F3,s𝐹2=2𝑥((𝑔𝑖𝐴)2+(𝑔𝑖𝑉)2)Σ𝑖𝑓𝑖ig*eq**2*data['P3'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))

Eq(F_1, \Sigma_{i}*e_{q}**2*f_i)

Eq(F_2, 2*x*\Sigma_{i}*e_{q}**2*f_i)

Eq(F_3, 0)

$\gamma Z$

In [30]:
display(sp.Eq(F1,sig*eq*data['P1'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
display(sp.Eq(F2,sig*eq*data['P2'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
display(sp.Eq(F3,sig*eq*data['P3'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))

Eq(F_1, 2*g_{V}^{i}*\Sigma_{i}*e_{q}*f_i)

Eq(F_2, 4*g_{V}^{i}*x*\Sigma_{i}*e_{q}*f_i)

Eq(F_3, 4*g_{A}^{i}*\Sigma_{i}*e_{q}*f_i)

$Z$

In [28]:
display(sp.Eq(F1,(sig*data['P1'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f).simplify()))
display(sp.Eq(F2,(sig*data['P2'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f)).simplify())
display(sp.Eq(F3,(sig*data['P3'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f)).simplify())

Eq(F_1, (g_{A}^{i}**2 + g_{V}^{i}**2)*\Sigma_{i}*f_i)

Eq(F_2, 2*x*(g_{A}^{i}**2 + g_{V}^{i}**2)*\Sigma_{i}*f_i)

Eq(F_3, 4*g_{A}^{i}*g_{V}^{i}*\Sigma_{i}*f_i)

# Delta Function (On-Shell condition for outgoing parton)

* $\displaystyle
\delta_{+}(p'^2) = \delta_{+}([q+p]^2) = \delta_{+}(g(\xi)) = \frac{1}{|g'(\xi_{\rm root})|}\delta_{+}(\xi - \xi_{\rm root})
$  
    * Note: $g(\xi) = 2p \cdot q - Q^2$ (massless partons)

In [ ]:
xi  = sp.symbols(r'\xi')
x   = sp.symbols(r'x')
Q   = sp.symbols(r'Q')
xhat = sp.symbols(r'\hat{x}')

g_xi    = xi*Q**2/x - Q**2
xi_root = sp.solve(sp.Eq(g_xi,0),xi)[0]
factor  = 1/sp.diff(g_xi,xi).subs(xi,xi_root)
xhat_   = x/xi_root

delta = sp.Function(r'\delta')
display(factor*delta(xi-xi_root))